## Student name: Firdosa mohamed

## Student pace: Hybrid

## Technical mentor : Maryann Mwikali

## Problem statement

The goal of this project is to create a prediction model that can identify SyriaTel customers who are prone to churn. Our goal is to identify predictable patterns and key features linked with customer churn. Specifically, we shall answer the following questions:

Are there any patterns among clients that stop utilizing SyriaTel services?
What features make a substantial contribution to client churn?

## Business Understanding

SyriaTel is a prominent mobile network operator in Syria.The company provides calls, news, messages, GSM, and internet services. The industry is now experiencing severe competition, thus client retention is critical for sustaining corporate revenue development. Customer churn study will assist us in identifying the factors that contribute to customer attrition. This will allow for the execution of steps that will assist reduce the number of clients who discontinue utilizing the company's services. The stakeholder audience here includes the data analytics team, sales team, senior management, and marketing executives.

## Objectives

1.Create a machine learning model that can effectively anticipate customer attrition based on previous customer data and behavioral patterns.

2.Identify the essential attributes that contribute significantly to client attrition.This will allow for the development of insights into the fundamental reasons.

3.To achieve a high level of accuracy and recall in the churn prediction model in order to reduce both false negatives (customers who churn but were not forecasted) and false positives (customers who were expected to churn but did not).

4.Utilize the prediction model to develop mitigation strategies to assist reduce customer turnover.

5.To make advice to the company on how to retain clients, which will assist the company keep its market share.

## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import zscore
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")